In [1]:
!pip install -U pyautogen openai dataset -qq

In [3]:
import autogen
import pprint
from autogen import ConversableAgent
from llm_config import llm_config


# chat between agents

In [5]:
doctor = ConversableAgent(
    name="john",
    system_message=
    "you are a medical doctor that advices patients based on symptoms.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

patient = ConversableAgent(
    name='smith',
    system_message=
    "You are a patient who went to doctors appointment, tell a doctor what you feel so you get advice from a doctor",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [6]:
chat_result = doctor.initiate_chat(
    recipient=patient, 
    message="Hello what can i help you with today?",
    max_turns=4,
)

john (to smith):

Hello what can i help you with today?

--------------------------------------------------------------------------------
smith (to john):

Hello doctor, thank you for seeing me today. I have been having a persistent cough for a week now and also experiencing shortness of breath. I feel very tired and have been having difficulty sleeping at night. Additionally, I have been experiencing body aches and pains, as well as a headache that won't go away. What do you think could be causing these symptoms and what should I do about them?

--------------------------------------------------------------------------------
john (to smith):

Hello, I'm sorry to hear that you are not feeling well. Given the symptoms you have described, it is possible that you may have a respiratory or viral infection. However, I cannot rule out the possibility of other underlying health conditions.  The cough, along with shortness of breath could be due to asthma, chronic obstructive pulmonary disease

In [7]:
pprint.pprint(chat_result.chat_history)

[{'content': 'Hello what can i help you with today?', 'role': 'assistant'},
 {'content': 'Hello doctor, thank you for seeing me today. I have been having '
             'a persistent cough for a week now and also experiencing '
             'shortness of breath. I feel very tired and have been having '
             'difficulty sleeping at night. Additionally, I have been '
             'experiencing body aches and pains, as well as a headache that '
             "won't go away. What do you think could be causing these symptoms "
             'and what should I do about them?',
  'role': 'user'},
 {'content': "Hello, I'm sorry to hear that you are not feeling well. Given "
             'the symptoms you have described, it is possible that you may '
             'have a respiratory or viral infection. However, I cannot rule '
             'out the possibility of other underlying health conditions.  The '
             'cough, along with shortness of breath could be due to asthma, '
      

# Agents for doing a task

In [76]:
machine_translation_agent = ConversableAgent(
    name="translation agent",
    system_message="""You are a helpful agent which is capable of doing machine translation 
    from any language to english,
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [77]:
sentiment_analysis_agent = ConversableAgent(
    name="Sentiment analysis agent",
    system_message="""You are a helpful agent given english text you can classify 
    into one of the sentiment classes. the options are 'positive' 'negative' or 'neutral'
    """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)



In [78]:
output_agent = ConversableAgent(
    name="Output Agent",
    system_message="""You are a helpful sentiment agent that tells the 
            the customer the setiment of their text and clear explanation why
            """,
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [79]:
input_agent = ConversableAgent(
    name="input_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [85]:
chats = [
    {
        "sender": machine_translation_agent,
        "recipient": input_agent,
        "message": 
            "Hello, I'm here to help you identify sentiment of a text from any language",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the translaton"
                             "into as JSON object only: "
                             "{'text': '', 'translation': ''}",
        },
        "max_turns": 1,
    },
    {
        "sender": machine_translation_agent, 
        "recipient": sentiment_analysis_agent,
        "summary_method": "reflection_with_llm",
        "message": 
            "i will take the json and work on creating a sentoment from it",
        "summary_args": {
            "summary_prompt" : "Return the translaton"
                             "into as JSON object only: "
                             "{'text': '', 'sentiment': ''}",
        },
        "max_turns": 1,
    },
    {
        "sender": sentiment_analysis_agent,
        "recipient": output_agent,
        "message": "here is what we know about your text.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

In [86]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)



********************************************************************************
Starting a new chat....

********************************************************************************
translation agent (to input_agent):

Hello, I'm here to help you identify sentiment of a text from any language

--------------------------------------------------------------------------------
input_agent (to translation agent):

ሰላም ጥሩ ነው

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
translation agent (to Sentiment analysis agent):

i will take the json and work on creating a sentoment from it
Context: 
Sure! Here's the translation and JSON object:

Text: ሰላም ጥሩ ነው
Translation: Hello, good morning

JSON Object: {"text": "ሰላም ጥሩ ነው", "translation": "Hello, good morning"}

--------

In [87]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")
    

Sure! Here's the translation and JSON object:

Text: ሰላም ጥሩ ነው
Translation: Hello, good morning

JSON Object: {"text": "ሰላም ጥሩ ነው", "translation": "Hello, good morning"}


Sure, here's the JSON object with just the translation and sentiment keys:

{"text": "Hello, good morning", "sentiment": "neutral"}


The text "ሰላም ጥሩ ነው" was translated to "Hello, good morning" and the sentiment of the translated text is neutral.


